Import the preprocessed ISOT dataset

In [125]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={'train': "preprocessed_isot/isot_train.csv", 'test': "preprocessed_isot/isot_test.csv"})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['statement', 'label'],
        num_rows: 8578
    })
    test: Dataset({
        features: ['statement', 'label'],
        num_rows: 16713
    })
})


Pick the pre-trained model

In [126]:
model_name = "albert/albert-base-v2"
your_path = 'isot_new_results'

Look over the label distribution

In [127]:
from collections import Counter

train_label_distribution = Counter(dataset['train']['label'])
test_label_distribution = Counter(dataset['test']['label'])

print("Training Label Distribution:", train_label_distribution)
print("Test Label Distribution:", test_label_distribution)

Training Label Distribution: Counter({False: 4310, True: 4268})
Test Label Distribution: Counter({True: 8528, False: 8185})


Labels in their original form are strings (true/false). We need to convert them to numerical values so they can be processed by the model.

In [128]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(dataset['train']['label'])

def encode_labels(example):
    return {'encoded_label': label_encoder.transform([example['label']])[0]}

for split in dataset:
    dataset[split] = dataset[split].map(encode_labels, batched=False)

The id2label and label2id mappings in AutoConfig are used to inform the model of the specific label-to-ID mappings so we can get the actual label names rather than the numerical reps when we do inference with the model.

In [129]:
from transformers import AutoConfig

unique_labels = sorted(list(set(dataset['train']['label'])))
id2label = {i: label for i, label in enumerate(unique_labels)}
label2id = {label: i for i, label in enumerate(unique_labels)}

config = AutoConfig.from_pretrained(model_name)
config.id2label = id2label
config.label2id = label2id


Verify the correct labels are being used

In [130]:
print("ID to Label Mapping:", config.id2label)
print("Label to ID Mapping:", config.label2id)

ID to Label Mapping: {0: False, 1: True}
Label to ID Mapping: {False: 0, True: 1}


Now, we need to tokenize the text data so it can be processed by the model. We will use the tokenizer that corresponds to the model we are using from Hugging Face's Transformers library.

In [131]:
from transformers import AlbertForSequenceClassification, AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertForSequenceClassification.from_pretrained(model_name, config=config)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We need to filter out any examples that have invalid content (e.g. empty strings) before tokenizing the data. Then, we will encode the labels and tokenize the text data. Tokenizing means converting the text data into numerical representations that can be processed by the model.

In [132]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['statement', 'label', 'encoded_label'],
        num_rows: 8578
    })
    test: Dataset({
        features: ['statement', 'label', 'encoded_label'],
        num_rows: 16713
    })
})


In [133]:
def filter_invalid_content(example):
    return isinstance(example['statement'], str)

dataset = dataset.filter(filter_invalid_content, batched=False)

def encode_data(batch):
    tokenized_inputs = tokenizer(batch["statement"], padding="max_length", truncation=True, max_length=256)
    tokenized_inputs["labels"] = batch["encoded_label"]
    return tokenized_inputs

dataset_encoded = dataset.map(encode_data, batched=True)

Verify the data is tokenized correctly:

In [134]:
print(dataset_encoded)

DatasetDict({
    train: Dataset({
        features: ['statement', 'label', 'encoded_label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8578
    })
    test: Dataset({
        features: ['statement', 'label', 'encoded_label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 16713
    })
})


In [135]:
dataset_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

The DataCollatorWithPadding ensures that all input sequences in a batch are padded to the same length, using the padding logic defined by the tokenizer. This is necessary because the model can only process inputs of the same length.

In [136]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

Next we'll set up LabelEncoder to encode labels and defines a function to compute per-label accuracy from a confusion matrix, providing label-specific accuracy metrics. I.e. when we train the model we want to see the accuracy metrics per label as well as the average metrics. This is more relevant if you have more than two labels, and one is underperforming. 

In [137]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  confusion_matrix
import numpy as np

label_encoder = LabelEncoder()
label_encoder.fit(unique_labels)

def per_label_accuracy(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    correct_predictions = cm.diagonal()
    label_totals = cm.sum(axis=1)
    per_label_acc = np.divide(correct_predictions, label_totals, out=np.zeros_like(correct_predictions, dtype=float), where=label_totals != 0)
    return dict(zip(labels, per_label_acc))

Compute the following metrics: accuracy, precision, recall, f1 score, and per-label accuracy.

In [138]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    decoded_labels = label_encoder.inverse_transform(labels)
    decoded_preds = label_encoder.inverse_transform(preds)

    precision = precision_score(decoded_labels, decoded_preds, average='weighted')
    recall = recall_score(decoded_labels, decoded_preds, average='weighted')
    f1 = f1_score(decoded_labels, decoded_preds, average='weighted')
    acc = accuracy_score(decoded_labels, decoded_preds)

    labels_list = list(label_encoder.classes_)
    per_label_acc = per_label_accuracy(decoded_labels, decoded_preds, labels_list)

    per_label_acc_metrics = {}
    for label, accuracy in per_label_acc.items():
        label_key = f"accuracy_label_{label}"
        per_label_acc_metrics[label_key] = accuracy

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        **per_label_acc_metrics
    }

Next, the training begins. Training loss and validation loss should decrease consistenly. If the training loss is decreasing but the validation loss is increasing, the model is overfitting. If both are increasing, the model is underfitting.

In [139]:
print(dataset_encoded['train'][0])


{'input_ids': tensor([    2, 13223,    15,  5766,     9,    13,     5,   139, 21458,    18,
            6,    13,     8,   132,   148,    72,  2632,    59,    46, 20252,
           35,    21,  5159,  7095,  6950,    19, 13223,    15,  4425,    15,
          236,  1617,    57,  5263,    21,  8022,   149,    14,  2235,  3389,
         2316,    15,  1148,    24,    13,     1,   108, 10424,    43,    21,
        10597,     9,     1, 10064,  4396,    15,    13, 22940,  1046,    13,
          103,  9967,  6401,    17,  1010,   334, 28357,  1346,  5263,    14,
         8022,    27,  8775,    19,  6811,  3343,   495,    19, 13223,    15,
         5865,    30,  7095,     1,    18,  4251,  3029,    16,    13,     1,
         3060,   105,    70,     1,    35,    21,  6950,    35,    14,  4425,
          294,  3087,   459,    27,   285,   137,    23,  2081,    20,  1744,
           33,  5955,    20,    13,     1,  3699, 21095,    15, 20297,  1825,
          558,    20,  4681, 14558,    15,   215, 

In [140]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=3,
    warmup_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=100,
    learning_rate=2e-5,
    save_steps=1000,
    gradient_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded['train'],
    eval_dataset=dataset_encoded['test'],
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

Save the results, the model and the state of the model.

In [ ]:
trainer.evaluate()
trainer.save_model(your_path)
trainer.save_state()

In [ ]:
import torch
torch.cuda.is_available()